# Modeling

### Import de Bibliotecas

In [ ]:
import pickle
import pathlib
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import GradientBoostingRegressor, RandomForestRegressor
from sklearn.metrics import mean_squared_error, make_scorer

### Preparação de Dados

In [91]:
DATA_DIR = pathlib.Path.cwd().parent / 'data'
clean_data_path = DATA_DIR / 'processed' / 'ames_clean.pkl'
with open(clean_data_path, 'rb') as file:
    data = pickle.load(file)

In [92]:
data.head()

,MS.SubClass,MS.Zoning,Lot.Frontage,Lot.Area,Lot.Shape,Land.Contour,Lot.Config,Land.Slope,Neighborhood,Bldg.Type,...,Sale.Type,Sale.Condition,SalePrice,Condition,HasShed,HasAlley,Exterior,Garage.Age,Remod.Age,House.Age
0,20,RL,141.0,31770.0,IR1,Lvl,Corner,Gtl,NAmes,1Fam,...,GroupedWD,Normal,5.332438,Norm,False,False,BrkFace,50.0,50.0,50.0
1,20,RH,80.0,11622.0,Reg,Lvl,Inside,Gtl,NAmes,1Fam,...,GroupedWD,Normal,5.021189,Roads,False,False,VinylSd,49.0,49.0,49.0
2,20,RL,81.0,14267.0,IR1,Lvl,Corner,Gtl,NAmes,1Fam,...,GroupedWD,Normal,5.235528,Norm,False,False,Wd Sdng,52.0,52.0,52.0
3,20,RL,93.0,11160.0,Reg,Lvl,Corner,Gtl,NAmes,1Fam,...,GroupedWD,Normal,5.387390,Norm,False,False,BrkFace,42.0,42.0,42.0
4,60,RL,74.0,13830.0,IR1,Lvl,Inside,Gtl,Gilbert,1Fam,...,GroupedWD,Normal,5.278525,Norm,False,False,VinylSd,13.0,12.0,13.0


In [93]:
X = data.drop('SalePrice', axis=1)
y = data['SalePrice']

In [94]:
X_model = pd.get_dummies(X, drop_first=True).copy()

### Divisão de Treino e Teste

In [95]:
X_train, X_test, y_train, y_test = train_test_split(
    X_model,
    y,
    test_size=0.2,
    random_state=42,
)

## Modelos (Treinamento e Hiperparâmetros)

### 1- Linear Regression

In [ ]:
# Initialize the Linear Regressor model
linear_model = LinearRegression()

# Fit Model to the training data
linear_model.fit(X_train, y_train)

# Crie uma função de scoring personalizada
scoring = make_scorer(mean_squared_error, greater_is_better=False)

# Perform 5-fold cross-validation on the linear model using negative MSE as the scoring metric.
cv_scores = cross_val_score(linear_model, X_model, y, cv=5, scoring=scoring)

### 2- Gradient Boosting Regressor

In [97]:
# Initialize the Gradient Boosting Regressor model
boosting_model = GradientBoostingRegressor()

# Define a parameter grid for tuning the model
param_grid = {
    'n_estimators': [100, 200, 300],    # Number of boosting stages
    'learning_rate': [0.01, 0.1, 0.2],  # Step size shrinkage
    'max_depth': [3, 5, 7],             # Maximum depth of each tree
    'subsample': [0.8, 1.0],            # Fraction of samples used for fitting individual base learners
}

# Initialize GridSearchCV with the boosting model
grid_search = GridSearchCV(estimator=boosting_model, param_grid=param_grid, cv=3, scoring='neg_mean_squared_error', n_jobs=-1, verbose=2)

# Fit GridSearchCV to the training data
grid_search.fit(X_train, y_train)

# Retrieve the best model and its parameters
best_boosting_model = grid_search.best_estimator_
print("Best Hyperparameters:", grid_search.best_params_)


Fitting 3 folds for each of 54 candidates, totalling 162 fits
Best Hyperparameters: {'learning_rate': 0.1, 'max_depth': 3, 'n_estimators': 300, 'subsample': 0.8}


Melhor Hiperparâmetros | Taxa de Aprendizagem | Profundidade Máx. | Número de etapas de reforço | Subsamples |
--- | --- | --- | --- | --- |
 1 | 0.1 | 3 | 300 | 0.8 |

### 3- Random Forest Regressor

In [98]:
# Initialize the Random Forest Regressor model
rf_model = RandomForestRegressor(random_state=42)

# Define a parameter grid for tuning the model
param_grid_rf = {
    'n_estimators': [100, 200, 300],  # Number of trees in the forest
    'max_depth': [10, 20, 30, None],  # Maximum depth of the tree
    'min_samples_split': [2, 5, 10],  # Minimum number of samples required to split an internal node
    'min_samples_leaf': [1, 2, 4],    # Minimum number of samples required to be at a leaf node
    'bootstrap': [True, False],       # Whether bootstrap samples are used when building trees
}

# Initialize GridSearchCV with the Random Forest model
grid_search_rf = GridSearchCV(estimator=rf_model, param_grid=param_grid_rf, cv=3, scoring='neg_mean_squared_error', n_jobs=-1, verbose=2)

# Fit GridSearchCV to the training data
grid_search_rf.fit(X_train, y_train)

# Retrieve the best model and its parameters
best_rf_model = grid_search_rf.best_estimator_
print("Best Hyperparameters:", grid_search_rf.best_params_)


Fitting 3 folds for each of 216 candidates, totalling 648 fits
Best Hyperparameters: {'bootstrap': True, 'max_depth': 20, 'min_samples_leaf': 1, 'min_samples_split': 2, 'n_estimators': 300}


Melhor Hiperparâmetros | Inicialização | Profundidade Máx. | Nº Mínimo Amostras Folha | Nº Mínimo Amostras | Número de árovres |
--- | --- | --- | --- | --- | --- |
 1 | True | 20 | 1 | 2 | 300 |

## Predição

#### Linear Regression

In [117]:
mean_mse = -cv_scores.mean()
rsme_ = np.sqrt(mean_mse)
print(f"MSE: {mean_mse:.6f} | RMSE: {rsme_:.6f}")

MSE: 0.003144 | RMSE: 0.056074


In [ ]:
errorlr = 100 * (10**rsme_ - 1)
print(f'Average error: {errorlr:.2f}%')

Average error: 13.78%


#### Gradient Boosting Regression

In [101]:
y_pred = best_boosting_model.predict(X_test)
mse = mean_squared_error(y_test, y_pred)
rsme_ = np.sqrt(mse)
print(f"MSE: {mse:.6f} | RMSE: {rsme_:.6f}")

MSE: 0.002463 | RMSE: 0.049625


In [102]:
errorgb = 100 * (10**rsme_ - 1)
print(f'Average error: {errorgb:.2f}%')

Average error: 12.11%


#### Random Forest Regression

In [103]:
y_pred_rf = best_rf_model.predict(X_test)
mse_rf = mean_squared_error(y_test, y_pred_rf)
rsme_ = np.sqrt(mse_rf)
print(f"MSE: {mse_rf:.6f} | RMSE: {rsme_:.6f}")

MSE: 0.003387 | RMSE: 0.058202


In [104]:
errorrf = 100 * (10**rsme_ - 1)
print(f'Average error: {errorrf:.2f}%')

Average error: 14.34%


## Escolha do Modelo

Após a análise dos resultados obtidos com os três modelos de regressão avaliados, a escolha do Gradient Boosting Regressor se destacou como a mais adequada para o projeto devido ao seu desempenho superior em todas as métricas principais. A tabela comparativa mostra os seguintes valores de MSE (Mean Squared Error), RMSE (Root Mean Squared Error) e erro percentual para cada modelo:

Index | Modelo | MSE | RMSE | Error |
--- | --- | --- | --- | --- |
 1 | Linear Regression | 0.003144 | 0.056074 | 13.78% |
 2 | Gradient Boosting Regressor | 0.002463 | 0.049625 | 12.11% |
 3 | Random Forest Regressor | 0.003387 | 0.058202 | 14.34% |

Motivo da Escolha do Gradient Boosting Regressor:

- Menor MSE: O Gradient Boosting Regressor apresentou um MSE de 0.002463, o mais baixo entre os três modelos testados, indicando uma menor média dos erros ao quadrado, o que reflete uma maior precisão nas previsões;

- Menor RMSE: O RMSE de 0.049625 do Gradient Boosting Regressor reforça seu bom desempenho, mostrando que o desvio entre os valores previstos e os valores reais foi o menor, comparado aos outros modelos;

- Erro Percentual mais Baixo: Com um erro percentual de 12.11%, o Gradient Boosting Regressor foi o mais preciso, superando a Regressão Linear (13.78%) e o Random Forest Regressor (14.34%);

- Equilíbrio entre Complexidade e Desempenho: Embora a Regressão Linear e o Random Forest Regressor apresentem resultados razoáveis, o Gradient Boosting Regressor se sobressai por combinar modelos simples de maneira otimizada, proporcionando previsões mais consistentes e um menor erro geral.

Portanto, com base na análise dessas métricas, o Gradient Boosting Regressor foi selecionado como o modelo mais adequado para o projeto, equilibrando precisão e robustez de maneira superior aos outros modelos avaliados.

### Impotância das Features e suas Implicações

In [ ]:
importancias = pd.DataFrame({'importancia': best_boosting_model.feature_importances_}, index=X_train.columns)

importancias_ordenadas = importancias.sort_values('importancia', ascending=False)

top_10_importantes = importancias_ordenadas.head(10)
print("Top 10 Features Mais Importantes:")
print(top_10_importantes)

Top 10 Features Mais Importantes:
               importancia
House.Age         0.264582
Gr.Liv.Area       0.220363
Garage.Cars       0.064484
Total.Bsmt.SF     0.060291
Fireplaces        0.045405
Garage.Area       0.035794
Exter.Qual_TA     0.033176
X1st.Flr.SF       0.031111
BsmtFin.SF.1      0.029465
Remod.Age         0.026067


Analisando as duas features com importância discrepante e suas possíveis implicações para a aplicação de negócios:

- House.Age (Idade da Casa) - Importância: 0.264582<br>
Implicação: A idade da casa tem uma grande influência no preço do imóvel. Casas mais novas tendem a ter um valor de mercado mais alto, já que provavelmente requerem menos manutenção e oferecem características mais modernas. Em um cenário de negócios, isso pode indicar que a empresa pode focar mais em propriedades novas ou reformadas, que podem ser mais atraentes para os compradores e gerar preços mais altos.

- Gr.Liv.Area (Área útil da casa - em metros quadrados) - Importância: 0.220363<br>
Implicação: A área útil da casa tem uma das maiores importâncias no preço, o que é intuitivo, já que casas maiores geralmente têm preços mais altos. Em termos de aplicação de negócios, isso sugere que oferecer imóveis maiores ou focar na maximização do espaço útil pode ser uma estratégia eficaz para aumentar o valor de mercado de uma propriedade. Além disso, pode influenciar as decisões de marketing e vendas, já que áreas maiores podem ser mais atraentes para famílias.

### Consequências do desempenho do modelo final para a aplicação de negócios

As consequências para a aplicação de negócios sugerem que o foco deve estar em imóveis mais novos e recentemente reformados, pois a idade da casa e das reformas são variáveis importantes para determinar o valor de mercado. 
- Imóveis mais novos ou com reformas recentes tendem a ter um valor mais alto, e realizar melhorias em imóveis antigos pode aumentar significativamente seu preço de venda. 
- A maximização do espaço é crucial, já que a área útil da casa e a área do porão são grandes determinantes do preço, indicando que aumentar o tamanho da propriedade ou otimizar o uso de espaços, como converter porões, pode agregar valor. 
- As comodidades adicionais, como lareiras e vagas de garagem, também são muito valorizadas pelos compradores, o que implica que incorporar mais vagas ou lareiras pode ser uma boa estratégia de marketing para justificar preços mais altos. 
- A qualidade da construção, especialmente a aparência externa da casa, é um ponto de venda importante, e investir na melhoria do exterior pode aumentar significativamente o valor percebido do imóvel.

De forma geral, as features mostram que o valor do imóvel em Ames, Iowa, é fortemente influenciado pela idade da casa, pelo espaço útil, pelas comodidades (como garagem e lareiras) e pela qualidade da construção. Isso oferece orientações valiosas para ajustar a oferta de imóveis e as estratégias de marketing, além de fornecer insights sobre como otimizar o preço das propriedades.